In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC

import sklearn.manifold as skm 
import sklearn.utils.random as skr # for random sampling 


In [25]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    # Early stopping 
    epsilon = 0.001
    counter = 0
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        if len(train_acc) % 5 == 0:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            break
        if max(test_acc) - test_acc[-1] >= epsilon:
            counter += 1
            if counter >= 20:
                sys.stderr.write("Early stopping initiated")
                print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
                break
        else:
            counter = 0
    return np.asarray(train_acc), np.asarray(test_acc)

In [4]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename+".pkl", "wb"), compress=True)
    file = open(filename+".txt", "a")
    msg = str(100*max(train_acc)) + " " + str(100*max(test_acc)) + " " +\
        str(len(train_acc)) + " " + str(np.argmax(test_acc) + 1) + "\n"
    file.write(msg)
    file.close()

In [19]:
############### ISOMAP ########################
# Everyone uses a neighborhood size of 8-12.
# If your manifold is more than 2-3 dimensional, then Isomap probably won't work, 
# you would need to have a neighborhood size that is larger (more points!) so that 
# you have reasonable options to estimate your geodesic path with links to nearby neighbors.
def iso_wrapper(n_neighbors, n_components):
    
    start = time.time()
    isomap = skm.Isomap(n_neighbors, n_components)
    isomap.fit(manifolddata)
    end = time.time()
    # Preparation time 
    prep_time = int(end - start) 
    print( "Prep time: ", prep_time ) 
    
    start = time.time()
    trainiso = isomap.transform(traindata)
    testiso = isomap.transform(testdata)
    end = time.time() 
    # Transformation time
    trans_time = int(end-start)
    print( "Transform time: ", trans_time )

    filename = "./dumper/isomap_data_"+str(n_neighbors)+"_"+str(n_components)+".pkl"
    joblib.dump((trainiso, trainlabels, testiso, testlabels), open(filename, "wb"), compress=True)
    
    return trainiso, testiso, prep_time, trans_time

In [17]:
############### Modified/Hessian Locally Linear Embedding , more ########################

def lle_wrapper(n_neighbors, n_components, lle_type = "modified"):
    
    start = time.time()
    mlle = skm.LocallyLinearEmbedding(n_neighbors, n_components, method = lle_type)
    mlle.fit(manifolddata)
    end = time.time()
    prep_time = int(end - start) 
    print( prep_time ) 

    start = time.time()
    trainmlle = mlle.transform(traindata)
    testmlle = mlle.transform(testdata)
    end = time.time()
    trans_time = int(end-start)
    print( trans_time )

    file_name = "./dumper/"+lle_type+"_data_"+str(n_neighbors)+"_"+str(n_components)+".pkl"
    joblib.dump((trainmlle, trainlabels, testmlle, testlabels), open(filename, "wb"), compress=True)

    return trainiso, testiso, prep_time, trans_time

In [7]:
############### Spectral Embedding ########################
############### Multi-dimensional Scaling ########################
############### t-distributed Stochastic Neighbor Embedding #######
# No online learning version; has to input full data

In [8]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()

Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [9]:
# data cropping
traindata = traindata[:20000]
testdata = testdata[:5000]

In [10]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'MNIST', 'dataset': 'MNIST', 'D': 2500, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 5, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 250, 'nFeatures': 784, 'nClasses': 10}


In [ ]:
# Run manifolding learning data
# Random sampling for manifold 
manifoldlist = skr.sample_without_replacement(len(traindata), 2000)
manifolddata = np.asfarray(traindata[np.asarray(manifoldlist)])
n_neighbors = 50
n_components  = 25
param["nFeatures"] = n_components
traintrans, testtrans, prep_time, trans_time = iso_wrapper(n_neighbors, n_components)

In [14]:
# Run basiline data 
# param["nFeatures"] = nFeatures
# traintrans, testtrans = traindata, testdata

In [11]:
################# VANILLA #################
def vanilla_train(param):
    hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    # Update param with bid
    param = hdb.getParam()
    print(bid)

    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traintrans)
    #trainencoded = hde.encodeData(traindata)
    HDE.saveEncoded(trainencoded, trainlabels, bid, "train")
    testencoded = hde.encodeData(testtrans)
    #testencoded = hde.encodeData(testdata)
    HDE.saveEncoded(testencoded, testlabels, bid, "test")

    # Should have 95%
    for i in range(param["iter_per_encoding"]):
        hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
        train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
        
        filename = "./logfile/baseline_"+str(param["D"])+"_"+str(param["id"])
        dump_log(param, train_acc, test_acc, filename)

In [12]:
# Dimension tuning 
param["D"] = 500

In [ ]:
vanilla_train(param)

for i in range(param["iter_per_trial"] - 1):
    
    # Run manifolding learning data
    # Random sampling for manifold 
    manifoldlist = skr.sample_without_replacement(len(traindata), 2000)
    manifolddata = np.asfarray(traindata[np.asarray(manifoldlist)])
    n_neighbors = 50
    n_components  = 25
    param["nFeatures"] = n_components
    traintrans, testtrans, prep_time, trans_time = iso_wrapper(n_neighbors, n_components)
    
    vanilla_train(param)

Generating vanilla HD basis of shape... 


1897


(500, 25)
Encoding time: 0.028887510299682617 
Dumping basis into file: base_1897.pkl 
Encoding data of shape (20000, 25)


Time spent: 56 sec
Dumping data into encoded_1897_train.pkl 
Encoding data of shape (5000, 25)


Time spent: 12 sec
Dumping data into encoded_1897_test.pkl 
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


...

Train: 0.819000 	 	 Test: 0.723000


Early stopping initiated

Train: 0.817700 	 	 Test: 0.744200
Train: 0.824500 	 	 Test: 0.726800
Train: 0.824500 	 	 Test: 0.726800


Early stopping initiatedEarly stopping initiated

Train: 0.821850 	 	 Test: 0.736600
